##1. Import Package & Load Data

In [0]:
import pandas as pd
import json
from tqdm import tqdm
import datetime
import numpy as np
import re
import yaml

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
link_biz = 'https://drive.google.com/open?id=1AL0wDIuJAxjST4f6LzTsq7JIw_h5emKy'
_, drive_id_biz = link_biz.split('=')
downloaded_biz = drive.CreateFile({'id': drive_id_biz})
downloaded_biz.GetContentFile('./business.json')
business = pd.read_json('./business.json', lines=True)
print(len(business))
business.head()

192609


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [0]:
link = 'https://drive.google.com/open?id=1udmP7qxjcpUqIMtA5kH5Hp2Vt2uTVRKQ'
_, drive_id = link.split('=')
downloaded = drive.CreateFile({'id': drive_id})
downloaded.GetContentFile('./sample_reviews.csv')
reviews = pd.read_csv('./sample_reviews.csv')
print(len(reviews))
reviews.head()

953504


,user_id,business_id,rating,date,reviews,useful,funny,cool
0,jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,3.0,2016-05-07 01:21:02,Tracy dessert had a big name in Hong Kong and ...,5.0,4.0,5.0
1,5JVY32_bmTBfIGpCCsnAfw,Mem13A3C202RzT53npn4NA,5.0,2017-05-13 10:41:43,If you are looking for the best pierogies in P...,9.0,6.0,9.0
2,XPZVfP7DQCSL3Nb9t2vxsA,1HD5iUUfVJDbfEBIn9yVhw,5.0,2017-03-15 02:02:13,Yes... the Boba Tea explosion is in full force...,11.0,8.0,11.0
3,hTZb1INhnCG8URaFywVoYQ,nDaW2hhQV5KYiGH7HzAOcg,3.0,2016-12-04 02:34:48,I stumbled upon this ice cream store and seein...,1.0,0.0,1.0
4,v7FPnMzdbl6J7U_8H1BWZA,zk3AimWhuz0EZsNgIddKdg,1.0,2018-01-09 21:27:45,"Awful place. Staff is not nice, very rude. Th...",1.0,0.0,0.0


In [0]:
print('Unique User', len(reviews['user_id'].unique()),  'Unique Business', len(reviews['business_id'].unique()))

Unique User 45890 Unique Business 109775


## One Hot Encoding for Categories

- Ignore business that is closed

In [0]:
business = business[business['is_open'] == 1].reset_index(drop=True)

In [0]:
# some businesses dont have categories, so we fill NaN.
business['categories'] = business['categories'].fillna('other')
business['categories_split'] = business['categories'].apply(lambda x: x.split(','))

In [0]:
# we would like to find a set of categories
list_ = []
for i in range(len(business['categories_split'])): 
    for j in range(len(business['categories_split'][i])):
        list_.append(business['categories_split'][i][j].strip(' ')) 
category = list(set(list_))

- Do one_hot encoding for category

In [0]:
def one_hot_encoding(category, data):
    char_to_int = dict((c, i) for i, c in enumerate(category))
    int_to_char = dict((c, 0) for i, c in enumerate(category))
    integer_encoded = [char_to_int[char.strip(' ')] for char in data]
    letter = [0 for _ in range(len(category))]
    for value in integer_encoded:
        letter[value] = 1
    return letter

In [0]:
list2 = []
for i in business['categories_split']:
    list2.append(one_hot_encoding(category, i))

In [0]:
catogories_one_hot_encoding = pd.DataFrame(np.array(list2), columns = category)
catogories_one_hot_encoding.head()

,Web Design,Laundromat,Cabinetry,Appliances & Repair,Spin Classes,Carpet Cleaning,Scavenger Hunts,Plastic Surgeons,Tanning,African,Flyboarding,Vehicle Wraps,Home Health Care,Cantonese,Nicaraguan,Electricity Suppliers,Skating Rinks,Pop-Up Restaurants,Reunion,Pet Boarding,Tax Services,Brazilian,Hot Air Balloons,Vinyl Records,Legal Services,Glass Blowing,Greek,House Sitters,Diagnostic Services,Shoe Shine,Eatertainment,Poutineries,Personal Care Services,Cards & Stationery,Comfort Food,Cocktail Bars,Japanese Curry,Currency Exchange,Brasseries,Makeup Artists,...,Clothing Rental,Food Court,Hindu Temples,Tours,Tattoo Removal,Art Classes,Venezuelan,Szechuan,Piano Stores,Music & DVDs,Supper Clubs,Reflexology,Taekwondo,Emergency Rooms,Salvadoran,Car Brokers,Mediterranean,Trivia Hosts,Hang Gliding,Propane,Home Organization,Head Shops,Jewelry Repair,Hair Removal,Wholesalers,Knife Sharpening,Stadiums & Arenas,Active Life,Personal Shopping,Flight Instruction,Breweries,Travel Agents,Restaurants,Creperies,Speech Therapists,Roofing,Pets,DIY Auto Shop,Podiatrists,Uniforms
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


- Join business id

In [0]:
catogories_one_hot_encoding['business_id'] = business['business_id']

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
catogories_one_hot_encoding.to_csv('/content/drive/My Drive/personalization/bus_categories.csv')

## One Hot Encoding for Attributes

In [0]:
business['attributes_keys'] = business['attributes'].apply(lambda x: None if x is None else x.keys())

- Get the parents attribute list 

In [0]:
att_list = []
for i in business['attributes_keys']:
    if i is not None:
        for j in list(i):
            att_list.append(j)

In [0]:
# from the list to set
print(len(set(att_list)), set(att_list))

39 {'DogsAllowed', 'RestaurantsTableService', 'OutdoorSeating', 'WheelchairAccessible', 'GoodForMeal', 'Open24Hours', 'Music', 'Smoking', 'DriveThru', 'Alcohol', 'BikeParking', 'HasTV', 'BYOB', 'HairSpecializesIn', 'DietaryRestrictions', 'RestaurantsPriceRange2', 'RestaurantsReservations', 'Corkage', 'AcceptsInsurance', 'GoodForDancing', 'BusinessAcceptsBitcoin', 'Caters', 'RestaurantsDelivery', 'GoodForKids', 'AgesAllowed', 'CoatCheck', 'RestaurantsGoodForGroups', 'RestaurantsTakeOut', 'RestaurantsAttire', 'WiFi', 'ByAppointmentOnly', 'HappyHour', 'RestaurantsCounterService', 'BestNights', 'Ambience', 'NoiseLevel', 'BusinessAcceptsCreditCards', 'BYOBCorkage', 'BusinessParking'}


- Get the childen attribute list

In [0]:
# the attributes are formatted as dictionary, 
skey = []
for i in business['attributes']:
    if i is not None:
        for key, value in i.items():
            if '{' in str(value):
                skey.append(key)

In [0]:
skey = set(skey)
print(len(skey), set(att_list))

7 {'DogsAllowed', 'RestaurantsTableService', 'OutdoorSeating', 'WheelchairAccessible', 'GoodForMeal', 'Open24Hours', 'Music', 'Smoking', 'DriveThru', 'Alcohol', 'BikeParking', 'HasTV', 'BYOB', 'HairSpecializesIn', 'DietaryRestrictions', 'RestaurantsPriceRange2', 'RestaurantsReservations', 'Corkage', 'AcceptsInsurance', 'GoodForDancing', 'BusinessAcceptsBitcoin', 'Caters', 'RestaurantsDelivery', 'GoodForKids', 'AgesAllowed', 'CoatCheck', 'RestaurantsGoodForGroups', 'RestaurantsTakeOut', 'RestaurantsAttire', 'WiFi', 'ByAppointmentOnly', 'HappyHour', 'RestaurantsCounterService', 'BestNights', 'Ambience', 'NoiseLevel', 'BusinessAcceptsCreditCards', 'BYOBCorkage', 'BusinessParking'}


In [0]:
def r_attribute(att):
    if att is not None:
        keys = att.keys()
        for i in list(keys):
            if i in skey:
                if att.get(i) != 'None':
                    for subkey, subvalue in yaml.load(att.get(i)).items():
                        att[i + '(' + subkey + ')'] = str(subvalue) 
                del att[i]
        return att

In [0]:
business['r_attribute'] = business['attributes'].apply(lambda x: r_attribute(x))

In [0]:
nkey = []
for i in business['r_attribute']:
    if i is not None:
        for key, value in i.items():
            if value == 'True' or value == 'False' or value == 'None':
                pass
            else:
                nkey.append(key)
nkey = set(nkey)

In [0]:
nkey -= {'RestaurantsAttire', 'RestaurantsPriceRange2','BYOBCorkage', 'AgesAllowed'}

- change the format of attribute to Parents(child)

In [0]:
def r_attribute2(att):
    if att is not None:
        keys = att.keys()
        for key in list(keys):
            if key in nkey:
                value = att.get(key)
                value = value.replace("u'", '').replace("'", '')
                att[key + '(' + value + ')'] = 'True'
                del att[key]
    return att

In [0]:
business['r_attribute2'] = business['r_attribute'].apply(lambda x: r_attribute2(x))

In [0]:
new_att_list = []
for i in business['r_attribute2']:
    if i is not None:
        for j in list(i):
            new_att_list.append(j)
new_att_list = set(new_att_list)

In [0]:
char_to_int = dict((c, i) for i, c in enumerate(new_att_list))
l = []
for i in {'RestaurantsAttire', 'RestaurantsPriceRange2','BYOBCorkage', 'AgesAllowed'}:
    l.append(char_to_int[i])

In [0]:
def one_hot_encoding_attribute(attribute, data):
    
    char_to_int = dict((c, i) for i, c in enumerate(attribute))
    int_to_char = dict((c, 0) for i, c in enumerate(attribute))
    letter = [0 for _ in range(len(attribute))]
    if data is not None:
        integer_encoded = [char_to_int[char.strip(' ')] for char in data]
        for value in integer_encoded:
            if value in l:
                letter[value] = data[list(attribute)[value]]
            else:
                letter[value] = 1
    return letter

In [0]:
attributes_one_hot_encoding = pd.DataFrame(list3, columns = list(new_att_list))

- remove the columns for the same meaning

In [0]:

attributes_one_hot_encoding['WiFi(no)'] = attributes_one_hot_encoding['WiFi(None)'] + attributes_one_hot_encoding['WiFi(no)']
attributes_one_hot_encoding['Smoking(no)'] = attributes_one_hot_encoding['Smoking(no)']+ attributes_one_hot_encoding['Smoking(None)']
attributes_one_hot_encoding['Alcohol(no)'] = attributes_one_hot_encoding['Alcohol(None)']+ attributes_one_hot_encoding['Alcohol(none)']
attributes_one_hot_encoding = attributes_one_hot_encoding.drop(['WiFi(None)','Smoking(None)', 'Alcohol(None)', 'Alcohol(none)'], axis=1)
attributes_one_hot_encoding = attributes_one_hot_encoding.drop(['BestNights(monday)', 'BestNights(tuesday)','BestNights(wednesday)','BestNights(thursday)','BestNights(friday)','BestNights(saturday)','BestNights(sunday)'],axis=1)
attributes_one_hot_encoding = attributes_one_hot_encoding.drop(['RestaurantsAttire','BYOBCorkage', 'AgesAllowed'], axis = 1)

In [0]:
attributes_one_hot_encoding.head()

,Alcohol(beer_and_wine),WheelchairAccessible,Ambience(classy),WiFi(free),Music(jukebox),BestNights(thursday),BikeParking,BusinessParking(garage),BusinessParking(validated),NoiseLevel(quiet),NoiseLevel(very_loud),GoodForMeal(breakfast),DietaryRestrictions(halal),Music(background_music),DietaryRestrictions(gluten-free),HairSpecializesIn(africanamerican),Caters,NoiseLevel(None),HairSpecializesIn(coloring),HairSpecializesIn(perms),RestaurantsGoodForGroups,GoodForMeal(dinner),BestNights(tuesday),BestNights(sunday),RestaurantsTakeOut,DietaryRestrictions(vegan),Music(video),Ambience(trendy),HairSpecializesIn(straightperms),RestaurantsCounterService,BusinessAcceptsCreditCards,RestaurantsTableService,OutdoorSeating,BestNights(saturday),HairSpecializesIn(kids),DietaryRestrictions(dairy-free),Ambience(upscale),GoodForMeal(lunch),GoodForMeal(brunch),AgesAllowed,...,DietaryRestrictions(vegetarian),Smoking(no),AcceptsInsurance,BusinessAcceptsBitcoin,RestaurantsDelivery,BestNights(monday),NoiseLevel(loud),RestaurantsAttire,ByAppointmentOnly,HappyHour,Music(karaoke),BusinessParking(street),DogsAllowed,Ambience(touristy),GoodForMeal(dessert),Open24Hours,Ambience(intimate),Music(dj),HairSpecializesIn(extensions),HasTV,Ambience(hipster),BYOB,DietaryRestrictions(kosher),RestaurantsPriceRange2,RestaurantsReservations,HairSpecializesIn(curly),Smoking(yes),Music(no_music),Smoking(outdoor),WiFi(paid),GoodForDancing,GoodForKids,CoatCheck,BusinessParking(lot),Ambience(romantic),GoodForMeal(latenight),Corkage,DietaryRestrictions(soy-free),BusinessParking(valet),Alcohol(no)
0,0,0,1,0,0,0,1,1,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,1,1,0,...,0,0,0,0,1,0,1,u'casual',0,0,0,1,0,1,1,0,1,0,0,1,1,0,0,2,1,0,0,0,0,0,0,1,0,1,1,1,0,0,1,0
1,1,0,1,0,0,0,1,1,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,1,0,0,1,1,1,0,0,0,1,1,1,0,...,0,0,0,0,1,0,0,'casual',0,0,0,1,0,1,1,0,1,0,0,1,1,0,0,2,1,0,0,0,0,0,0,1,0,1,1,1,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [0]:
attributes_one_hot_encoding.to_csv('/content/drive/My Drive/personalization/bus_attributes.csv')

## 4. Business Side Info
- concat categories and attributes

In [0]:
business_u = business[['business_id', 'stars', 'review_count']]
bus_one = pd.concat([business_u, attributes_one_hot_encoding, catogories_one_hot_encoding], axis = 1)
bus_one.to_csv('/content/drive/My Drive/personalization/bus_sideinfo.csv')